In [1]:
import numpy as np
import pandas as pd
from sodapy import Socrata
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
%matplotlib inline

In [2]:
#Make API call to City of Chicago Open Data Portal
#Include only inspections prior to 3/31/2021
client = Socrata("data.cityofchicago.org", None)
results = client.get("4ijn-s7e5", where="inspection_date < '2021-04-01T00:00:00.000'", limit=100000000)

inspections_df = pd.DataFrame.from_records(results)
inspections_df.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2484567,ROYALTY,ROYALTY,1306130,Restaurant,Risk 1 (High),3810 W 63RD ST,CHICAGO,IL,60629,2021-02-05T00:00:00.000,Non-Inspection,No Entry,41.778836516734856,-87.71836138998039,"{'latitude': '-87.71836138998039', 'longitude'...",NaN
1,2473041,ROSATI'S GRANT PARK,ROSATI'S,2762683,Restaurant,Risk 1 (High),23 E ADAMS ST,CHICAGO,IL,60603,2021-01-22T00:00:00.000,License,Pass,41.879391313239694,-87.62684825563626,"{'latitude': '-87.62684825563626', 'longitude'...",NaN
2,2464198,FOOD FIRST CHICAGO,FOOD FIRST CHICAGO,2762429,Catering,Risk 1 (High),741 S WESTERN AVE,CHICAGO,IL,60612,2020-12-28T00:00:00.000,License,Pass,41.871853705220325,-87.68603684594997,"{'latitude': '-87.68603684594997', 'longitude'...",NaN
3,2463977,HWA WON,HWA WON,2341742,Restaurant,Risk 1 (High),2519 W PETERSON AVE,CHICAGO,IL,60659,2020-12-21T00:00:00.000,Non-Inspection,No Entry,41.99036795715765,-87.6929808527407,"{'latitude': '-87.6929808527407', 'longitude':...",NaN
4,2463667,M & M FOOD,M & M FOOD,1193207,NaN,Risk 3 (Low),7073 N WESTERN AVE,CHICAGO,IL,60645,2020-12-11T00:00:00.000,Canvass,Out of Business,42.01076851593425,-87.69006746560828,"{'latitude': '-87.69006746560828', 'longitude'...",NaN


In [3]:
inspections_df["inspection_type"].unique()

array(['Non-Inspection', 'License', 'Canvass', 'Canvass Re-Inspection',
       'Complaint', 'License Re-Inspection', 'Short Form Complaint',
       'Complaint Re-Inspection', 'Recent Inspection',
       'Suspected Food Poisoning', 'Consultation', nan,
       'License-Task Force', 'Suspected Food Poisoning Re-inspection',
       'Not Ready', 'Task Force Liquor 1475', 'Complaint-Fire',
       'Short Form Fire-Complaint', 'Pre-License Consultation',
       'Out of Business', 'Tag Removal', 'Special Events (Festivals)',
       'No Entry', 'OUT OF BUSINESS', 'out ofbusiness',
       'Business Not Located', 'COVID COMPLAINT', 'OFFICE ASSIGNMENT',
       'Recent inspection', 'fire complaint', 'FIRE',
       'Complaint-Fire Re-inspection', 'KITCHEN CLOSED FOR RENOVATION',
       'CORRECTIVE ACTION', 'O.B.', 'LICENSE CANCELED BY OWNER',
       'OWNER SUSPENDED OPERATION/LICENSE', 'LICENSE CONSULTATION',
       'License consultation', 'Illegal Operation', 'Package Liquor 1474',
       'No entry'

In [4]:
#Take only re-inspections
inspections_df['inspection_type_clean'] = inspections_df['inspection_type'].str.lower()
inspections_df['inspection_type_clean'] = inspections_df['inspection_type_clean'].str.replace('-', '')

#inspections_df[inspections_df['inspection_type_clean'].isna()]
#One record is missing inspection type; will drop
inspections_df = inspections_df.dropna(subset=['inspection_type_clean'])
#inspections_df['inspection_type_clean'].unique()

reinspections_df = inspections_df[inspections_df['inspection_type_clean'].str.contains('reinspection')]
reinspections_df.shape

(41455, 18)

In [5]:
inspections_df.shape

(218229, 18)

In [6]:
#matching steps:
#merge full inspections_df with reinspections_df
#delete where inspection_id = reinspection_id
#group by ID of reinspection
#within each group, retain only inspection with closest date before (but not equal to) reinspection
full_merge = inspections_df.merge(reinspections_df, on=['dba_name', 'address', 'license_', 'latitude', 'longitude'],
                                    suffixes=('_orig', '_re'))
full_merge.shape

(494272, 31)

In [7]:
full_merge.columns

Index(['inspection_id_orig', 'dba_name', 'aka_name_orig', 'license_',
       'facility_type_orig', 'risk_orig', 'address', 'city_orig', 'state_orig',
       'zip_orig', 'inspection_date_orig', 'inspection_type_orig',
       'results_orig', 'latitude', 'longitude', 'location_orig',
       'violations_orig', 'inspection_type_clean_orig', 'inspection_id_re',
       'aka_name_re', 'facility_type_re', 'risk_re', 'city_re', 'state_re',
       'zip_re', 'inspection_date_re', 'inspection_type_re', 'results_re',
       'location_re', 'violations_re', 'inspection_type_clean_re'],
      dtype='object')

In [8]:
full_merge_clean = full_merge[full_merge['inspection_id_orig'] != full_merge['inspection_id_re']]
full_merge_clean.shape

(452817, 31)

In [9]:
full_merge_clean['date_orig'] = pd.to_datetime(full_merge_clean['inspection_date_orig'])
full_merge_clean['date_re'] = pd.to_datetime(full_merge_clean['inspection_date_re'])

<ipython-input-9-2341921e68a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_merge_clean['date_orig'] = pd.to_datetime(full_merge_clean['inspection_date_orig'])
<ipython-input-9-2341921e68a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_merge_clean['date_re'] = pd.to_datetime(full_merge_clean['inspection_date_re'])


In [10]:
#remove rows where original inspection occurred after re-inspection
full_merge_clean['time_between'] = full_merge_clean['date_orig'] - full_merge_clean['date_re']
full_merge_clean['time_between'] = full_merge_clean['time_between'].dt.days
full_merge_clean = full_merge_clean[full_merge_clean['time_between'] <= 0]
full_merge_clean.shape

<ipython-input-10-6563173d44f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_merge_clean['time_between'] = full_merge_clean['date_orig'] - full_merge_clean['date_re']
<ipython-input-10-6563173d44f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_merge_clean['time_between'] = full_merge_clean['time_between'].dt.days


(248086, 34)

In [11]:
#full_merge_grp = full_merge_clean.groupby('inspection_id_re')
grped = full_merge_clean[full_merge_clean['time_between'] == full_merge_clean.groupby('inspection_id_re')['time_between'].transform('max')]

grped.head()

,inspection_id_orig,dba_name,aka_name_orig,license_,facility_type_orig,risk_orig,address,city_orig,state_orig,zip_orig,...,zip_re,inspection_date_re,inspection_type_re,results_re,location_re,violations_re,inspection_type_clean_re,date_orig,date_re,time_between
16,577275,ROYALTY,ROYALTY,1306130,Restaurant,Risk 1 (High),3810 W 63RD ST,CHICAGO,IL,60629,...,60629,2011-05-24T00:00:00.000,Canvass Re-Inspection,Pass,"{'latitude': '-87.71836138998039', 'longitude'...",33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,canvass reinspection,2011-04-18,2011-05-24,-36
28,2200477,HWA WON,HWA WON,2341742,Restaurant,Risk 1 (High),2519 W PETERSON AVE,CHICAGO,IL,60659,...,60659,2018-08-10T00:00:00.000,Canvass Re-Inspection,Pass w/ Conditions,"{'latitude': '-87.6929808527407', 'longitude':...","3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",canvass reinspection,2018-08-02,2018-08-10,-8
43,1506367,HWA WON,HWA WON,2341742,Restaurant,Risk 1 (High),2519 W PETERSON AVE,CHICAGO,IL,60659,...,60659,2015-02-24T00:00:00.000,License Re-Inspection,Pass,"{'latitude': '-87.6929808527407', 'longitude':...",NaN,license reinspection,2015-02-17,2015-02-24,-7
53,2320554,SUBWAY #44541,SUBWAY,1963876,Restaurant,Risk 1 (High),211 W ADAMS ST,CHICAGO,IL,60606,...,60606,2019-11-05T00:00:00.000,Canvass Re-Inspection,Pass,"{'latitude': '-87.63410319837826', 'longitude'...",51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...,canvass reinspection,2019-10-28,2019-11-05,-8
57,2176834,SUBWAY #44541,SUBWAY,1963876,Restaurant,Risk 1 (High),211 W ADAMS ST,CHICAGO,IL,60606,...,60606,2018-06-14T00:00:00.000,Canvass Re-Inspection,Pass,"{'latitude': '-87.63410319837826', 'longitude'...",NaN,canvass reinspection,2018-06-06,2018-06-14,-8


In [12]:
grped.shape

(41599, 34)

In [13]:
duplicates = grped[grped.duplicated('inspection_id_re')]
print(duplicates.shape)
duplicates[['inspection_id_orig', 'inspection_date_orig', 'inspection_id_re', 'inspection_date_re', 'time_between']].head(20)

(262, 34)


,inspection_id_orig,inspection_date_orig,inspection_id_re,inspection_date_re,time_between
2042,1966455,2016-10-14T00:00:00.000,1970656,2016-11-01T00:00:00.000,-18
3647,2252450,2019-01-10T00:00:00.000,2252797,2019-01-18T00:00:00.000,-8
8664,519218,2011-07-26T00:00:00.000,517448,2011-08-02T00:00:00.000,-7
19050,2285617,2019-04-23T00:00:00.000,2290882,2019-06-04T00:00:00.000,-42
46092,1760212,2016-05-19T00:00:00.000,1931711,2016-05-26T00:00:00.000,-7
46349,2144588,2018-02-06T00:00:00.000,2144892,2018-02-13T00:00:00.000,-7
46541,2050307,2017-05-12T00:00:00.000,2050414,2017-05-15T00:00:00.000,-3
50681,1965764,2016-09-30T00:00:00.000,1965767,2016-10-03T00:00:00.000,-3
58609,2213707,2018-08-30T00:00:00.000,2222313,2018-09-13T00:00:00.000,-14
69806,1591629,2015-12-11T00:00:00.000,1609295,2016-01-05T00:00:00.000,-25


In [14]:
grped['results_orig'].unique()

array(['Fail', 'Pass w/ Conditions', 'Pass', 'No Entry', 'Not Ready',
       'Out of Business'], dtype=object)

In [15]:
grped.groupby('results_orig').count()

,inspection_id_orig,dba_name,aka_name_orig,license_,facility_type_orig,risk_orig,address,city_orig,state_orig,zip_orig,...,zip_re,inspection_date_re,inspection_type_re,results_re,location_re,violations_re,inspection_type_clean_re,date_orig,date_re,time_between
results_orig,,,,,,,,,,,,,,,,,,,,,
Fail,37848,37848,37414,37846,37821,37848,37848,37815,37839,37841,...,37841,37848,37848,37848,37719,24820,37848,37848,37848,37848
No Entry,343,343,338,343,343,343,343,343,343,343,...,343,343,343,343,343,254,343,343,343,343
Not Ready,70,70,70,70,70,70,70,70,70,70,...,70,70,70,70,68,36,70,70,70,70
Out of Business,301,301,298,301,299,301,301,301,301,301,...,301,301,301,301,300,176,301,301,301,301
Pass,598,598,593,598,598,598,598,598,598,598,...,598,598,598,598,597,279,598,598,598,598
Pass w/ Conditions,2439,2439,2430,2439,2439,2439,2439,2437,2439,2438,...,2438,2439,2439,2439,2434,1804,2439,2439,2439,2439


In [16]:
grped_clean = grped[(grped['results_orig'] == 'Fail') | (grped['results_orig'] == 'Pass w/ Conditions')]
grped_clean.shape

(40287, 34)

In [17]:
duplicates2 = grped_clean[grped_clean.duplicated('inspection_id_re')]
print(duplicates2.shape)
duplicates2[['inspection_id_orig', 'inspection_date_orig', 'inspection_id_re', 'inspection_date_re', 'time_between']].head(20)

(69, 34)


,inspection_id_orig,inspection_date_orig,inspection_id_re,inspection_date_re,time_between
2042,1966455,2016-10-14T00:00:00.000,1970656,2016-11-01T00:00:00.000,-18
8664,519218,2011-07-26T00:00:00.000,517448,2011-08-02T00:00:00.000,-7
46092,1760212,2016-05-19T00:00:00.000,1931711,2016-05-26T00:00:00.000,-7
50681,1965764,2016-09-30T00:00:00.000,1965767,2016-10-03T00:00:00.000,-3
83223,88599,2010-11-19T00:00:00.000,88604,2010-11-23T00:00:00.000,-4
87753,580616,2011-04-27T00:00:00.000,580732,2011-05-04T00:00:00.000,-7
89965,1516051,2015-01-16T00:00:00.000,1516204,2015-01-23T00:00:00.000,-7
94596,1493729,2014-08-13T00:00:00.000,1441491,2014-08-21T00:00:00.000,-8
104666,80288,2010-05-03T00:00:00.000,80327,2010-06-17T00:00:00.000,-45
106750,580192,2011-04-04T00:00:00.000,580179,2011-04-04T00:00:00.000,0


In [18]:
#Dropping anything that's been duplicated. This only drops 69 rows. 
grped_no_duplicates = grped_clean.drop_duplicates(subset='inspection_id_re')
grped_no_duplicates = grped_no_duplicates.drop_duplicates(subset='inspection_id_orig')
grped_no_duplicates.shape

(40187, 34)

In [19]:
grped_no_duplicates['time_between'].describe()

count    40187.000000
mean       -11.147685
std         21.085523
min      -1121.000000
25%        -10.000000
50%         -7.000000
75%         -7.000000
max          0.000000
Name: time_between, dtype: float64

In [20]:
grped_no_duplicates.groupby('results_re').count()

,inspection_id_orig,dba_name,aka_name_orig,license_,facility_type_orig,risk_orig,address,city_orig,state_orig,zip_orig,...,state_re,zip_re,inspection_date_re,inspection_type_re,location_re,violations_re,inspection_type_clean_re,date_orig,date_re,time_between
results_re,,,,,,,,,,,,,,,,,,,,,
Fail,3512,3512,3433,3511,3509,3512,3512,3507,3512,3511,...,3512,3511,3512,3512,3495,3434,3512,3512,3512,3512
No Entry,340,340,335,340,340,340,340,340,340,340,...,340,340,340,340,339,259,340,340,340,340
Not Ready,36,36,36,36,36,36,36,36,36,36,...,36,36,36,36,36,27,36,36,36,36
Out of Business,81,81,81,81,80,81,81,81,81,81,...,81,81,81,81,81,14,81,81,81,81
Pass,31026,31026,30688,31025,31007,31026,31026,30998,31019,31020,...,31019,31020,31026,31026,30925,17802,31026,31026,31026,31026
Pass w/ Conditions,5192,5192,5172,5192,5188,5192,5192,5190,5190,5191,...,5190,5191,5192,5192,5177,5056,5192,5192,5192,5192


In [21]:
grped2 = grped_no_duplicates[(grped_no_duplicates['results_re'] == 'Fail') | (grped_no_duplicates['results_re'] == 'Pass') | (grped_no_duplicates['results_re'] == 'Pass w/ Conditions') ]
grped2['time_between'].describe()

count    39730.000000
mean       -11.013869
std         20.032972
min       -685.000000
25%        -10.000000
50%         -7.000000
75%         -7.000000
max          0.000000
Name: time_between, dtype: float64

In [22]:
final_grped = grped2[grped2['time_between'] >= -150]

In [23]:
final_grped.columns

Index(['inspection_id_orig', 'dba_name', 'aka_name_orig', 'license_',
       'facility_type_orig', 'risk_orig', 'address', 'city_orig', 'state_orig',
       'zip_orig', 'inspection_date_orig', 'inspection_type_orig',
       'results_orig', 'latitude', 'longitude', 'location_orig',
       'violations_orig', 'inspection_type_clean_orig', 'inspection_id_re',
       'aka_name_re', 'facility_type_re', 'risk_re', 'city_re', 'state_re',
       'zip_re', 'inspection_date_re', 'inspection_type_re', 'results_re',
       'location_re', 'violations_re', 'inspection_type_clean_re', 'date_orig',
       'date_re', 'time_between'],
      dtype='object')

In [24]:
final_df = final_grped[['inspection_id_orig', 'dba_name', 'license_', 
                        'facility_type_orig', 'date_orig', 'inspection_type_orig',
                        'results_orig', 'violations_orig', 'inspection_id_re',
                        'date_re', 'results_re', 'time_between']]
final_df = final_df.rename({'inspection_id_orig': 'id_orig', 'dba_name': 'name', 'license_': 'license', 
                        'facility_type_orig': 'facility_type', 'inspection_id_re': 'id_re'}, axis=1)

In [25]:
print(final_df.shape)
final_df.head()

(39570, 12)


,id_orig,name,license,facility_type,date_orig,inspection_type_orig,results_orig,violations_orig,id_re,date_re,results_re,time_between
16,577275,ROYALTY,1306130,Restaurant,2011-04-18,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,577343,2011-05-24,Pass,-36
28,2200477,HWA WON,2341742,Restaurant,2018-08-02,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",2200718,2018-08-10,Pass w/ Conditions,-8
43,1506367,HWA WON,2341742,Restaurant,2015-02-17,License,Fail,"10. SEWAGE AND WASTE WATER DISPOSAL, NO BACK S...",1506378,2015-02-24,Pass,-7
53,2320554,SUBWAY #44541,1963876,Restaurant,2019-10-28,Canvass,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,2321003,2019-11-05,Pass,-8
57,2176834,SUBWAY #44541,1963876,Restaurant,2018-06-06,Canvass,Fail,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,2181359,2018-06-14,Pass,-8


In [26]:
#Drop conditions:
#repeat re-inspection ID or original inspection ID (may investigate further, but not a big impact)
#original outcome of pass, no entry, or not ready
#more than 150 days between original and reinspection
#went from 41,455 rows to 39,570

#save to pickle
final_df.to_pickle("initial_clean.pkl")